# Helper Functions 
(Run hidden cell)
- Training Accuracy and Loss Graphs:    plot_history(history)
- Predictions:                          print_predictions(inceptionv3_model, test_ds)
- True and Predicted Classes:           true_classes,predicted_classes = true_pred_classes(inceptionv3_model, test_ds)
- Accuracy:                             accuracy_score(true_classes,predicted_classes)
- F1 Score:                             f1_score(true_classes, predicted_classes, average='weighted')
- Recall:                               recall_score(true_classes, predicted_classes, average='weighted')
- Precision:                            precision_score(true_classes, predicted_classes, average='weighted')
- Confusion MAtrix (as Array):          conf_matrix = confusion_matrix(true_classes, predicted_classes)
- Plot Confusion Matrix:                print_conf_matrix(true_classes, predicted_classes,class_names)
- Upload a trained model to azure:      download_model_from_azure("model_name")
- Download a trained model from azure:  upload_model_to_azure(model, "model_name")

In [65]:
image_size=224
channels=3
autotune = tf.data.experimental.AUTOTUNE # Adapt preprocessing and prefetching dynamically

def data_split(df):
    """Splits and returns the dataset into training, validation, and test"""
    X_temp, X_test, y_temp, y_test = train_test_split(df['Image_Folder'], df['Label'], test_size=0.15, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.176, random_state=42)
    #convert labels to array
    y_train = np.array(y_train).tolist()
    y_val = np.array(y_val).tolist()
    y_test = np.array(y_test).tolist()
    #print number of observations per datasets
    print("Nr. Training:",len(X_train),"Nr. Validation:",len(X_val),"Nr. Test:",len(X_test))
    
    return X_train, X_val, X_test, y_train, y_val, y_test


def load_image(image_path):
    """Fetches image data from Azure Blob Storage."""
    try:
        # Create a blob service client
        blob_service_client = BlobServiceClient(account_url="https://<account_name>.blob.core.windows.net", credential="<account_key>")
        blob_client = container_client.get_blob_client(image_path)

        # Download the blob's contents as bytes
        blob_data = blob_client.download_blob().readall()
        
        return blob_data
    
    except Exception as e:
        print(f"Error fetching image from Azure: {e}")
        # Return a default image or handle the error appropriately
        return tf.zeros((image_size, image_size, channels), dtype=tf.uint8).numpy()

def load_and_preprocess_image(path):
    """Loads an image, decodes it to grayscale, resizes, and normalizes it."""
    # Load image
    image_file = load_image(path)
    # Decode the image to grayscale
    image_tensor = tf.io.decode_image(image_file, channels=channels)
    # Resize the image
    image_resized = tf.image.resize(image_tensor, [image_size, image_size])
    # Normalize the image data
    image_normalized = image_resized / 255.0
    return image_normalized.numpy()

def create_data_lists(filenames, labels):
    """Creates lists of images and labels."""
    X_train = []
    y_train = []

    for path, label in zip(filenames, labels):
        image = load_and_preprocess_image(path)
        X_train.append(image)
        y_train.append(label)
    
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    
    # Convert labels to one-hot encoding
    #y_train = tf.keras.utils.to_categorical(y_train, num_classes=7)
    
    return X_train, y_train


def plot_history(model):
    """Plots the accuracy and loss of the inputted model."""
    # summarize history for accuracy
    plt.plot(model.history['accuracy'])
    plt.plot(model.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()
    
    # summarize history for loss
    plt.plot(model.history['loss'])
    plt.plot(model.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()


def print_predictions(labels, preds, predictions):
    """Predictions based on test dataset."""
    #predict
    for label,pred, prediction in zip(labels, preds, predictions):
        
            print("Prediction:", prediction,"Pred. Class: ",pred, "Actual Label:", label)# Print the first prediction
        
    
        
def plot_model(model): 
    """Plot model with predefined arguments."""
    plot_model(model, 
            to_file='vgg.png',
            show_shapes=True,
            show_dtype=True,
            show_layer_names=True,
            show_layer_activations=True,
            show_trainable=False)
    

def evaluate_model(model, test_ds):
    result = model.evaluate(test_ds)
    # Assuming accuracy was the second metric (index 1), extract the accuracy.
    test_accuracy = result[1] * 100  # Convert to percentage
    print(f"Test Accuracy: {test_accuracy:.2f}%")
    return test_accuracy


def true_and_predicted_classes(predictions, labels):
    predicted_classes = []
    true_classes = []
    for prediction in predictions:  
        pred = np.argmax(prediction)
        predicted_classes.append(pred)
    for label in labels: 
        true = np.argmax(label)
        true_classes.append(true)
    return predicted_classes, true_classes


def print_conf_matrix(true_classes, predicted_classes, class_names):
    """
    Print confusion matrix.
    """
    conf_matrix = confusion_matrix(true_classes, predicted_classes)
    df_cm = pd.DataFrame(
        conf_matrix, index=class_names, columns=class_names,
    )
    fig = plt.figure(figsize=(10,7))
    heatmap = sns.heatmap(df_cm, annot=True, fmt="d", cmap='Blues')

    # Set aesthetics for better readability
    heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=14)
    heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=14)

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.title('Confusion Matrix')
    plt.show()

def plot_image(index, images, labels):
    """
    Plots a single image with its predicted and actual labels.
    
    Parameters:
    - image: The image data to plot.
    - y_true: The true label of the image.
    - y_pred: The predicted label of the image.
    - class_names: List of class names for labels.
    """
    # Get a single image from X_val
    image = images[index]

    # Get the true label for the image
    y_true = np.argmax(labels[index])

    # Predict the label using the model
    y_pred = np.argmax(model.predict(np.expand_dims(image, axis=0)))

    plt.figure(figsize=(3, 3))
    plt.imshow(image, cmap='bone')
    plt.xticks([])
    plt.yticks([])
    plt.xlabel(f"True: {class_names[y_true]}\nPred: {class_names[y_pred]}",
               color='black' if y_true == y_pred else 'red')
    plt.title("Image with Predicted and Actual Labels")
    plt.show()

def plot_wrong_pred_image(images, true_classes, predicted_classes):
    """
    Plots a single image with its predicted and actual labels.
    
    Parameters:
    - image: The image data to plot.
    - y_true: The true label of the image.
    - y_pred: The predicted label of the image.
    - class_names: List of class names for labels.
    """
    mismatch =  [i for i in range(len(true_classes)) if true_classes[i] != predicted_classes[i]]
    random_nr = random.choice(mismatch)
    # Get a single image from X_val
    image = images[random_nr]

    # Get the true label for the image
    y_true = true_classes[random_nr]

    # Predict the label using the model
    y_pred = predicted_classes[random_nr]

    plt.figure(figsize=(3, 3))
    plt.imshow(image, cmap='bone')
    plt.xticks([])
    plt.yticks([])
    plt.xlabel(f"True: {class_names[y_true]}\nPred: {class_names[y_pred]}",
               color='black' if y_true == y_pred else 'red')
    plt.title("Wrong predicted image")
    plt.show()


# Setup

Run the follwing cell to pip install the necerssary packages specified in the requirements.txt file.

In [21]:
pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


to push/pull form ucloud enter following code in terminal
-  git config --global user.name "FIRST_NAME LAST_NAME"
-  git config --global user.email "MY_NAME@example.com"


Importing the necessary packages

In [2]:
import os
import io
import tensorflow as tf
import numpy as np
from azure.storage.blob import BlobServiceClient, ContainerClient
from azure.core.exceptions import ResourceNotFoundError
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
import ast
import time
from keras.models import load_model
import tempfile

import pandas as pd
from tensorflow.keras import layers, models
import tensorflow_hub as hub
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model, Sequential
from keras.utils import plot_model
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, confusion_matrix
from keras.callbacks import Callback, ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

2024-05-15 11:56:22.117338: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-15 11:56:22.121301: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-15 11:56:22.170140: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-15 11:56:23.109896: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### Connect to Azure

In [3]:
#set up storage
connection_string = "DefaultEndpointsProtocol=https;AccountName=mlfinalexam5505462853;AccountKey=0c40lghglG5/GlNK9yujDQAgo38GKoS2I3DeC/g22hwAEIFANKpmC/TqOpRk4RCT1DbfNiHBFt72+AStB+PfUA==;EndpointSuffix=core.windows.net"
container_name = "meterml"

#create client
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
container_client = blob_service_client.get_container_client(container_name)

### Load Image Paths and Labels

In [4]:
import pandas as pd
import ast
import numpy as np
#get filepaths
df_train= pd.read_csv("FINAL_METER_ML_train_2000.csv")
df_val = pd.read_csv("FINAL_METER_ML_val.csv")
df_test = pd.read_csv("FINAL_METER_ML_test.csv")

# convert each string in the DataFrame to a list
df_train['Label'] = df_train['Label'].apply(ast.literal_eval).apply(np.array)
df_val['Label'] = df_val['Label'].apply(ast.literal_eval).apply(np.array)
df_test['Label'] = df_test['Label'].apply(ast.literal_eval).apply(np.array)


# convert each list in the DataFrame to a numpy array
df['Label'] = df['Label'].apply(np.array)

class_names=["CAFOs","Landfills","Mines","Negative","ProcessingPlants","RefineriesAndTerminals","WWTreatment"]

X_train = df_train['Image_Folder']
X_val = df_val['Image_Folder']
X_test = df_test['Image_Folder']


y_train = np.array(df_train['Label']).tolist()
y_val = np.array(df_val['Label']).tolist()
y_test = np.array(df_test['Label']).tolist()


In [7]:
X_train, y_train = create_data_lists(X_train, y_train)
X_val, y_val = create_data_lists(X_val, y_val)
X_test, y_test = create_data_lists(X_test, y_test)

# Save the array to a file
np.save('x_train.npy', X_train)
np.save('x_test.npy', X_test)
np.save('x_val.npy', X_val)



# Models

## ResNet50: 
- https://datagen.tech/guides/computer-vision/resnet-50/
- https://medium.com/@bravinwasike18/building-a-deep-learning-model-with-keras-and-resnet-50-9dd6f4eb3351
- https://medium.com/@ozgunhaznedar/image-classification-on-satellite-images-with-deep-learning-baa9813dde4e
- https://wandb.ai/mostafaibrahim17/ml-articles/reports/The-Basics-of-ResNet50---Vmlldzo2NDkwNDE2#step-4:-building-resnet-50-model

### Transfer Learning

In [40]:
#set variables
train_epochs = 30
tune_epochs = 10
total_epochs = train_epochs + tune_epochs
batch_size = 128

# import resnet model for transfer learning
rn50_base = tf.keras.applications.ResNet50(
    include_top = False,
    weights = "imagenet",
    input_shape=(224,224,3)
    )

# Freeze layers pf basemodel, so the pre-trained weights are fixed
for each_layer in rn50_base.layers:
        each_layer.trainable=False

# create sequential model
resnet_model = Sequential()

# Add output layers for finetuning
resnet_model.add(rn50_base)
resnet_model.add(Flatten()) #use flatten instead of GlobalAveragePooling2D as it may yield better results when enough data
resnet_model.add(Dense(512, activation='relu'))
resnet_model.add(Dense(7, activation='softmax'))

# Compile model
resnet_model.compile(optimizer= tf.keras.optimizers.Adam(),
                    loss= tf.keras.losses.CategoricalCrossentropy(), 
                    metrics = ['accuracy'])

#initializt time
t0 = time.time()

# Train model 
history = resnet_model.fit(X_train,
                        y_train,
                        validation_data = (X_val,y_val)), 
                        epochs = train_epochs, 
                        batch_size=batch_size, 
                        callbacks = [ReduceLROnPlateau(patience=5), EarlyStopping(patience=5)])
t1 = time.time()

#print time in seconds
print("Training time in seconds:", t1-t0)

#save model
resnet_model.save("resnet_model.keras")

# Plot model information
plot_history(history)


ValueError: Data cardinality is ambiguous. Make sure all arrays contain the same number of samples.'x' sizes: 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 

### Fine Tuning

In [26]:
#unfreeze last convolution layer for fine tuning
for each_layer in rn50_base.layers:
        each_layer.trainable=False
for layer in [l for l in rn50_base.layers if 'conv5' in l.name]:
   layer.trainable = True
   
for i, layer in enumerate(rn50_base.layers):
    print(i, layer.name, "-", layer.trainable)

0 input_layer_3 - False
1 conv1_pad - False
2 conv1_conv - False
3 conv1_bn - False
4 conv1_relu - False
5 pool1_pad - False
6 pool1_pool - False
7 conv2_block1_1_conv - False
8 conv2_block1_1_bn - False
9 conv2_block1_1_relu - False
10 conv2_block1_2_conv - False
11 conv2_block1_2_bn - False
12 conv2_block1_2_relu - False
13 conv2_block1_0_conv - False
14 conv2_block1_3_conv - False
15 conv2_block1_0_bn - False
16 conv2_block1_3_bn - False
17 conv2_block1_add - False
18 conv2_block1_out - False
19 conv2_block2_1_conv - False
20 conv2_block2_1_bn - False
21 conv2_block2_1_relu - False
22 conv2_block2_2_conv - False
23 conv2_block2_2_bn - False
24 conv2_block2_2_relu - False
25 conv2_block2_3_conv - False
26 conv2_block2_3_bn - False
27 conv2_block2_add - False
28 conv2_block2_out - False
29 conv2_block3_1_conv - False
30 conv2_block3_1_bn - False
31 conv2_block3_1_relu - False
32 conv2_block3_2_conv - False
33 conv2_block3_2_bn - False
34 conv2_block3_2_relu - False
35 conv2_block3_3_c

In [27]:
#compile the model with smaller learning rate
resnet_model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0001),
                    loss = tf.keras.losses.CategoricalCrossentropy(), 
                    metrics=['accuracy'])

#initialize timing
t0 = time.time()

# Train model 
history = resnet_model.fit(X_train,
                        y_train
                        validation_data = (X_val,y_val)), 
                        epochs=total_epochs, 
                        batch_size=batch_size, 
                        callbacks = [ReduceLROnPlateau(patience=5), EarlyStopping(patience=5)])
t1 = time.time()
print("Training time in seconds:", t1-t0)

#save trained model
resnet_model.save("resnet_model_tf.keras")

# Plot training plots
plot_history(history)

Epoch 1/40


2024-05-15 06:33:12.617525: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


165/165 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.6880 - loss: 1.7288

2024-05-15 06:43:01.728517: W tensorflow/core/framework/op_kernel.cc:1827] UNKNOWN: HttpResponseError: The specified account is disabled.
RequestId:0e4d209e-201e-0046-3b82-a67091000000
Time:2024-05-15T04:43:01.7181094Z
ErrorCode:AccountIsDisabled
Content: <?xml version="1.0" encoding="utf-8"?><Error><Code>AccountIsDisabled</Code><Message>The specified account is disabled.
RequestId:0e4d209e-201e-0046-3b82-a67091000000
Time:2024-05-15T04:43:01.7181094Z</Message></Error>
Traceback (most recent call last):

  File "/home/ucloud/.local/lib/python3.12/site-packages/tensorflow/python/ops/script_ops.py", line 268, in __call__
    return func(device, token, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^

  File "/home/ucloud/.local/lib/python3.12/site-packages/tensorflow/python/ops/script_ops.py", line 146, in __call__
    outputs = self._call(device, args)
              ^^^^^^^^^^^^^^^^^^^^^^^^

  File "/home/ucloud/.local/lib/python3.12/site-packages/tensorflow/python/ops/script_ops.py", line 15

UnknownError: Graph execution error:

Detected at node EagerPyFunc defined at (most recent call last):
<stack traces unavailable>
HttpResponseError: The specified account is disabled.
RequestId:ea6ed3da-301e-0075-4882-a62f3a000000
Time:2024-05-15T04:43:01.7762631Z
ErrorCode:AccountIsDisabled
Content: <?xml version="1.0" encoding="utf-8"?><Error><Code>AccountIsDisabled</Code><Message>The specified account is disabled.
RequestId:ea6ed3da-301e-0075-4882-a62f3a000000
Time:2024-05-15T04:43:01.7762631Z</Message></Error>
Traceback (most recent call last):

  File "/home/ucloud/.local/lib/python3.12/site-packages/tensorflow/python/ops/script_ops.py", line 268, in __call__
    return func(device, token, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^

  File "/home/ucloud/.local/lib/python3.12/site-packages/tensorflow/python/ops/script_ops.py", line 146, in __call__
    outputs = self._call(device, args)
              ^^^^^^^^^^^^^^^^^^^^^^^^

  File "/home/ucloud/.local/lib/python3.12/site-packages